Count up sweeps from standing variation (origin pre-optimum, fixation post optimum) vs. from new mutations (both post-optimum)

In [1]:
#This script requires that reduce.sh has been executed from the BoGPoster directory
#The purpose of this notebook is to execute taht big loop below and picke the output...
#%matplotlib inline
#import matplotlib
#import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
import glob
import pickle

In [2]:
ff=glob.glob('../*.fixations.h5')

In [3]:
PARAMS=[]
OUTCOMES=[]
for i in ff:
    fn=i
    i=i.strip('../')
    i=i.strip('.fixations.h5')
    i=i.split('_')
    PARAMS.append(dict(zip(i[0::2],i[1::2])))
    data=pd.read_hdf(fn)
    data.reset_index(inplace=True,drop=True)
    
    ##TAlly up classic vs standing sweeps
    STANDING=[]
    STANDINGSIZES=[]
    STANDINGORIGINS=[]
    STANDINGFTIMES=[]
    STANDINGLENGTHS=[]
    FO=[]
    NEW=[]
    NEWSIZES=[]
    NEWORIGINS=[]
    NEWFTIMES=[]
    NEWLENGTHS=[]
    dg=data.groupby(['rep'])
    for i in dg.groups:
        dgi=dg.get_group(i)
        g2=dgi.groupby(['pos','esize','origin'])
        standing=0
        new=0
        for j in g2.groups:
            fi = g2.get_group(j)
            o=fi.iloc[0]['origin']
            f=fi.iloc[len(fi.index)-1]['generation']
            e=fi.iloc[0]['esize']
            rep=fi.iloc[0]['rep']
            if o < 10000 and f >= 10000:
                standing += 1
                STANDINGSIZES.append({'esize':e,'rep':rep})
                STANDINGORIGINS.append({'origin':o,'rep':rep})
                STANDINGLENGTHS.append({'flen':(f-o+1),'rep':rep})
                STANDINGFTIMES.append({'ftime':(f),'rep':rep})
                #This if frequency when optimum shifted,
                #Based on a pd.DF slice...
                fo=fi[fi.generation==10000].iloc[0]['freq']
                FO.append({'fo':fo,'rep':rep})
            elif o > 10000 and f > 10000:
                new+=1
                NEWSIZES.append({'esize':e,'rep':rep})
                NEWORIGINS.append({'origin':o,'rep':rep})
                NEWLENGTHS.append({'flen':(f-o+1),'rep':rep})
                NEWFTIMES.append({'ftime':(f),'rep':rep})
        STANDING.append(standing)
        NEW.append(new)
    OUTCOMES.append( {'standing':STANDING,
                      'standing_esizes':STANDINGSIZES,
                      'standing_origins':STANDINGORIGINS,
                      'standing_ftimes':STANDINGFTIMES,
                      'standing_lengths':STANDINGLENGTHS,
                      'f0':FO,
                      'new':NEW,
                      'new_esizes':NEWSIZES,
                      'new_origins':NEWORIGINS,
                      'new_ftimes':NEWFTIMES,
                      'new_lengths':NEWLENGTHS
                      }
                    )


In [4]:
pfile=open('params.pickle','w')
pickle.dump(PARAMS,pfile)
pfile.close()

In [5]:
pfile=open('outcomes.pickle','w')
pickle.dump(OUTCOMES,pfile)
pfile.close()